# Importing Libraries

In [1]:
!pip install efficientnet -q
!pip install wandb --upgrade -q

In [2]:
import os
import glob
import wandb
import numpy as np
import pandas as pd

import tensorflow as tf
import efficientnet.tfkeras as efn

from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split

# Configuration

In [3]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy

In [4]:
COMPETITION_NAME = "vinbigdata-chest-xray-resized-png-1024x1024"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

Running on TPU: grpc://10.0.0.2:8470
Running on 8 replicas


In [5]:
class Config:
    IMG_SIZE = 512
    NUM_REPLICAS_IN_SYNC = strategy.num_replicas_in_sync
    BATCH_SIZE = 16 * NUM_REPLICAS_IN_SYNC
    GDC_DS_PATH = GCS_DS_PATH
    NUM_CLASSES = 15
    LR = 0.001 * NUM_REPLICAS_IN_SYNC
    EPOCHS = 10

In [6]:
config = Config()

# Loading data

In [7]:
image_paths = glob.glob('../input/vinbigdata-chest-xray-abnormalities-detection/train/*')
temp_df = pd.DataFrame()
temp_df['image_paths'] = image_paths
temp_df['image_id'] = temp_df['image_paths'].map(lambda x: x.split('/')[-1].split('.')[0])
temp_df.head(2)

,image_paths,image_id
0,../input/vinbigdata-chest-xray-abnormalities-d...,4d390e07733ba06e5ff07412f09c0a92
1,../input/vinbigdata-chest-xray-abnormalities-d...,289f69f6462af4933308c275d07060f0


In [8]:
df = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
df = pd.merge(df, temp_df, on='image_id')
df.head(2)

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,image_paths
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,../input/vinbigdata-chest-xray-abnormalities-d...
1,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R15,NaN,NaN,NaN,NaN,../input/vinbigdata-chest-xray-abnormalities-d...


In [9]:
np.sum(df.isna())

image_id           0
class_name         0
class_id           0
rad_id             0
x_min          31818
y_min          31818
x_max          31818
y_max          31818
image_paths        0
dtype: int64

In [10]:
class_names = {}
for i in df.class_id.unique():
    name = df[df['class_id'] == i].class_name.values[0]
    class_names[i] = name
    
class_names

{14: 'No finding',
 3: 'Cardiomegaly',
 10: 'Pleural effusion',
 11: 'Pleural thickening',
 0: 'Aortic enlargement',
 13: 'Pulmonary fibrosis',
 5: 'ILD',
 8: 'Nodule/Mass',
 9: 'Other lesion',
 7: 'Lung Opacity',
 6: 'Infiltration',
 4: 'Consolidation',
 2: 'Calcification',
 1: 'Atelectasis',
 12: 'Pneumothorax'}

In [11]:
df_train, df_valid = train_test_split(df, test_size=0.1, random_state=1234, stratify=df['class_id'])

# Creating dataset

In [12]:
def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.with_options(ignore_order)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [13]:
GCS_DS_PATH

'gs://kds-810d06acb6c0cc88e4248e5f8942bad22e7ba031bf4d46ffd6a3bf1f'

In [14]:
train_paths = GCS_DS_PATH + '/train/' + df_train['image_id'] + '.png'
valid_paths = GCS_DS_PATH + '/train/' + df_valid['image_id'] + '.png'

train_labels = df_train['class_id'].values
valid_labels = df_valid['class_id'].values

In [15]:
train_decoder = build_decoder(with_labels=True, target_size=(config.IMG_SIZE, config.IMG_SIZE), ext='png')
valid_decoder = build_decoder(with_labels=False, target_size=(config.IMG_SIZE, config.IMG_SIZE), ext='.png')

In [16]:
train_dataset = build_dataset(train_paths,
                              train_labels,
                              bsize = config.BATCH_SIZE,
                              decode_fn = train_decoder)

In [17]:
valid_dataset = build_dataset(valid_paths,
                             valid_labels,
                             bsize = config.BATCH_SIZE,
                             decode_fn = train_decoder,
                             repeat = False, 
                             shuffle = False, 
                             augment = False)

# Training the model

First we will traing only the last layer and then later we will finetune the whole model.

In [18]:
with strategy.scope():
    
    base_model = efn.EfficientNetB7(input_shape = (config.IMG_SIZE, config.IMG_SIZE, 3),
                                    weights = 'imagenet',
                                    include_top = False)
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(config.NUM_CLASSES, activation='softmax')
    ])
        
    model.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()],
        steps_per_execution = 32)

    model.summary()

258441216/258434480 [==============================] - 6s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 16, 16, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 15)                38415     
Total params: 64,136,095
Trainable params: 63,825,375
Non-trainable params: 310,720
_________________________________________________________________


In [19]:
model_ckpt     = tf.keras.callbacks.ModelCheckpoint('./efficientNetPretrained.h5', save_best_only=True)
lr_scheduler   = tf.keras.callbacks.ReduceLROnPlateau(patience=1, min_delta=0.05)
early_stopping = tf.keras.callbacks.EarlyStopping(min_delta=0.01)

In [20]:
history = model.fit(x = train_dataset,
                    validation_data = valid_dataset,
                    epochs = config.EPOCHS,
                    steps_per_epoch = len(df_train)//config.BATCH_SIZE,
                    callbacks = [model_ckpt, lr_scheduler, early_stopping])

Epoch 1/10
477/477 [==============================] - 1877s 4s/step - loss: 1.3962 - sparse_categorical_accuracy: 0.5582 - val_loss: 1.2549 - val_sparse_categorical_accuracy: 0.5948
Epoch 2/10
477/477 [==============================] - 361s 757ms/step - loss: 1.0931 - sparse_categorical_accuracy: 0.6278 - val_loss: 1.0593 - val_sparse_categorical_accuracy: 0.6456
Epoch 3/10
477/477 [==============================] - 361s 757ms/step - loss: 1.0314 - sparse_categorical_accuracy: 0.6426 - val_loss: 1.0710 - val_sparse_categorical_accuracy: 0.6337


# Saving the model

In [21]:
featurizer = model.layers[:-1]
featurizer

In [22]:
temp_model = tf.keras.Sequential([
    featurizer[0],
    featurizer[1]
])

In [23]:
temp_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 16, 16, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
Total params: 64,097,680
Trainable params: 63,786,960
Non-trainable params: 310,720
_________________________________________________________________


In [24]:
temp_model.save('./feature_extractor.h5')